1. add_newgame_dir 제작
- 파일 경로 중 Data - Game_package_data - [Game_name] 만드는 역할.
- Data - Game_list.txt 에서 정보 읽고 패키지 정보 저장 위치 저장함.

2. save_calculation 제작
- 효율 계산한 결과 Game_package_data - [Game_name] 에 저장하는 역할.

In [1]:
import os
import pandas as pd
import numpy as np

file = pd.read_csv("Data\Game_list.txt", sep=",", encoding="UTF8")

file

,Null,Genshin_impact,Punishing_gray_raven,temp_game
0,0,Data\Game_package_data\Genshin_impact,Data\Game_package_data\Punishing_gray_raven,D:\Documents\Desktop\준상\Efficiency_calculator\...


편의 위해 함수 제작

In [10]:
from pandas import DataFrame

def read_csv(filepath : str) : 
    """Read csv file as UTF8 encoding.

    Args:
        filepath (str) : directory of csv file.

    Returns:
        pd.read_csv(filepath, sep=",", encoding="UTF8")
    """
    return pd.read_csv(filepath, sep=",", encoding="UTF8")
def save_csv(Data : DataFrame, filepath : str) : 
    """Save csv file as UTF8 encoding.

    Args:
        Data (DataFrame) : data build by DataFrame
        filepath (str) : directory of csv file

    Returns:
        Data.to_csv(filepath, sep=",", encoding="UTF8", index=False)
    """
    return Data.to_csv(filepath, sep=",", encoding="UTF8", index=False)

In [3]:
def return_data_dir(Game_name : str) -> str: 
    """Returns [Game_name]'s data directory.
    
    - Data - Game_package_data - Game_name

    Args:
        Game_name (str) : name of game

    Returns:
        pd.read_csv("Data\Game_list.txt", sep=",", encoding="UTF8")[Game_name].values[0] 
            --> Data - Game_package_data - Game_name
    """
    return pd.read_csv("Data\Game_list.txt", sep=",", encoding="UTF8")[Game_name].values[0]

In [4]:
def add_newgame_dir(Data_dir : str, Game_name : str) -> None: 
    """Only creates new game directory and saves to Game_list.txt
    
    If some game's package data directory is already exist, function do nothing.

    Creates : 
    - Current directory
        - Data
            - Game_package_data
                - Game_name
    
    Args:
        Data_dir (str) : relative or absolute directory of "Data"
        Game_name (str) : name of game
    """
    
    game_path = Data_dir + "\Game_package_data"
    
    game_list = read_csv(Data_dir + "\Game_list.txt")
    
    if Game_name in os.listdir(game_path) : 
        print(f"{Game_name}'s data looks like already exist at {game_path}")
        print(os.listdir(game_path))
        return
    
    game_dir = game_path + "\\" + Game_name
    os.mkdir(game_dir)
    
    if Game_name in game_list.keys() : 
        game_list = game_list.drop([Game_name], axis=1)
    
    game_list = pd.concat([game_list, 
                           pd.Series(data=game_dir, name=Game_name)
                           ], axis=1)
    
    save_csv(Data=game_list, filepath=Data_dir + "\Game_list.txt")
    print(f"({Game_name}) data directory has been created.")

In [5]:
def save_calculation(Calc_dataframe : DataFrame, Game_name : str, Force=False) -> None: 
    """Save pacakge's calculation
    
    Directory : 
    - Data
        - Game_package_data
            - Game_name
                - Calculation_Game_name.txt

    Args:
        Calc_dataframe (DataFrame) : calculated DataFrame
        Game_name (str) : name of game
        Force (bool) : force to save data though already exist
    """
    
    game_dir = return_data_dir(Game_name)
    
    if type(Calc_dataframe) != type(DataFrame()) : 
        print(f"Args Calc_dataframe must be dataframe. --> [ Calc_dataframe : {type(Calc_dataframe)} ]")
        return
    
    game_dir = game_dir + "\Calculation_" + Game_name + ".txt"
    
    if os.path.isfile(game_dir) : 
        print(f"{Game_name}'s data looks like already exist at {game_dir}")
        if not Force :
            return
        print("Forcing save data...")
    
    save_csv(Data=Calc_dataframe, filepath=game_dir)
    print(f"({Game_name}) calculation data has been saved.")

In [6]:
def save_package_data(Package_dataframe : DataFrame, Game_name : str, Force=False) -> None: 
    """Save package data.

    Args:
        Package_dataframe (DataFrame) : package data.
        Game_name (str) : name of game.
        Force (bool, optional) : force to save data though already exist. Defaults to False.
    """
    
    game_dir = return_data_dir(Game_name=Game_name)
    
    if type(Package_dataframe) != type(DataFrame()) : 
        print(f"Args Package_dataframe must be dataframe. --> [ Package_dataframe : {type(Package_dataframe)} ]")
        return
    
    game_dir = game_dir + "\Package_data.txt"
    
    if os.path.isfile(game_dir) : 
        print(f"{Game_name}'s data looks like already exist at {game_dir}")
        if not Force :
            return
        print("Forcing save data...")
    
    save_csv(Data=Package_dataframe, filepath=game_dir)
    print(f"({Game_name}) package data has been saved.")

In [58]:
def add_package_dataframe(Package_dataframe : DataFrame, Game_name : str) -> None: 
    """Add package data.

    Args:
        Package_dataframe (DataFrame): package data.
        Game_name (str): name of game.
    """
    
    game_dir = return_data_dir(Game_name=Game_name)
    
    package_dir = game_dir + "\Package_data.txt"
    
    prev_data = read_csv(package_dir).values
    new_data = Package_dataframe.values
    
    data_stack = np.vstack((prev_data, new_data))
    current_data = pd.DataFrame(data=data_stack, columns=Package_dataframe.keys())
    
    save_csv(current_data, package_dir)
    print(f"({Game_name}) package data has been created.")

(temp_game) package data has been created.


In [117]:
def delete_package(Game_name : str, Package_name : str, Force=False) : 
    """Delete specific package.

    Args:
        Game_name (str): name of game.
        Package_name (str): name of package.
        Force (bool, optional): force to delete every data. Defaults to False.
    """
    game_dir = return_data_dir(Game_name=Game_name)
    
    package_dir = game_dir + "\Package_data.txt"
    
    package_data = read_csv(package_dir)
    
    # print(package_data)
    del_index = package_data.index[package_data['name'] == Package_name]
    
    if len(del_index) > 1 : 
        print(f"There are multiple packages that named [ {Package_name} ] in data.")
        if not Force : 
            return
        print("Forcing to delete data...")
    
    del_index = sorted(del_index, reverse=True)
    
    for index in del_index : 
        package_data = package_data.drop(index=index)
    
    save_csv(package_data, package_dir)
    print(f"({Game_name}) package data ({Package_name}) has been deleted.")

In [119]:
delete_package("temp_game", "third", Force=True)

There are multiple packages that named [ third ] in data.
Forcing to delete data...
(temp_game) package data (third) has been deleted.
